### Testing and Checking the Bert Model 
In this notebook we load the SCIML Arxiv data set and test the Bert Model saved in the container.

In [1]:
import sys
import pandas as pd
import numpy as np
import pickle
import json
import os

#### Load the data
The follow set of functions load the data.  the files are in the folder classifiers/sci_doc.  There are two files there.   sciml_data_ariv.p and convig_all4 

In [2]:
root = '/'
def load_docs(path, name):
    filename = path+name+".p"
    fileobj = open(filename, "rb")
    z = fileobj.read()
    lst = pickle.loads(z)
    titles = []
    sitenames = []
    abstracts = []
    for i in range(0, len(lst)):
        titles.extend([lst[i][0]])
        sitenames.extend([lst[i][1]])
        abstracts.extend([lst[i][2]])
        
    #print("done loading "+filename)
    return abstracts, sitenames, titles


def read_config(subj, basepath):
    docpath =basepath+ "/config_"+subj+".json"
    with open(docpath, 'rb') as f:
        doc = f.read() 
    z =json.loads(doc)
    subject = z['subject']
    loadset = z['loadset']
    subtopics = []
    for w in z['supertopics']:
        subtopics.extend([(w[0], w[1])])
    return subject, loadset, subtopics

def make_dict(subtopics):
    dic = {}
    for main in subtopics:
        sl = main[1]
        for x in sl:
            dic[x] = main[0]
    return dic

def split_titles(disp_title):
    subject,loadset, subtopics = read_config("all4",root+"classifier/sci_doc")
    dic = make_dict(subtopics)
    lis = []
    for ti in disp_title:
        l = ti.find('[')
        if(l >= 0):
            #lis.append(ti[:l])
            e = ti[l+1:]
            l2 = e.find(']')
            e = e[:l2]
            try:
                if dic[e]== 'compsci':
                    lis.append([ti[:l], 0, e])
                if dic[e]== 'math':
                    lis.append([ti[:l], 1, e])
                if dic[e]== 'Physics':
                    lis.append([ti[:l], 2, e])
                if dic[e]== 'bio':
                    lis.append([ti[:l], 3, e])
                if dic[e]== 'finance':
                    lis.append([ti[:l], 4, e])
            except:
                e = 1
            
    return lis

####  Getting the data
there are three items that are loaded

abstracts - the text of the abstracts

sitenames - the names of the arxiv categories

disp_titles - titles of the docs

In [11]:
root = './'
abstracts, sitenames, disp_title = load_docs(root+"classifier/sci_doc/", "sciml_data_arxiv")


In [12]:
abstracts[0]

'Deep convolutional neural networks (CNN) have recently been shown in many computer vision and pattern recog- nition applications to outperform by a significant margin state- of-the-art solutions that use traditional hand-crafted features. However, this impressive performance is yet to be fully exploited in robotics. In this paper, we focus one specific problem that can benefit from the recent development of the CNN technology, i.e., we focus on using a pre-trained CNN model as a method of generating an image representation appropriate for visual loop closure detection in SLAM (simultaneous localization and mapping). We perform a comprehensive evaluation of the outputs at the intermediate layers of a CNN as image descriptors, in comparison with state-of-the-art image descriptors, in terms of their ability to match images for detecting loop closures. The main conclusions of our study include: (a) CNN-based image representations perform comparably to state-of-the-art hand- crafted compet

In [13]:
sitenames[0]

'cs.RO'

In [15]:
disp_title[0]

'Convolutional Neural Network-Based Image Representation for Visual Loop   Closure Detection [cs.RO]'

split_text will create list of pairs (abstract, true_index) where true_index is based on the topic of the astract.  the key is

0 = computer science

1 = math

2 = physics 

3 = bio

4 = finance



In [16]:
def split_text(disp_title, abstracts):
    subject,loadset, subtopics = read_config("all4",root+"/classifier/sci_doc")
    dic = make_dict(subtopics)
    lis = []
    ind = 0
    for ind in range(len(disp_title)): #disp_title,titles:
        ti = disp_title[ind]
        te = abstracts[ind]
        l = ti.find('[')
        if(l >= 0):
            #lis.append(ti[:l])
            e = ti[l+1:]
            l2 = e.find(']')
            e = e[:l2]
            try:
                if dic[e]== 'compsci':
                    lis.append([te, 0])
                if dic[e]== 'math':
                    lis.append([te, 1])
                if dic[e]== 'Physics':
                    lis.append([te, 2])
                if dic[e]== 'bio':
                    lis.append([te, 3])
                if dic[e]== 'finance':
                    lis.append([te, 4])
            except:
                e = 1
            
    return lis

#### training and test sets
we use list elements 0-4499 as the training set and 4500-7109 as the test set.  we are not training here so we really only need the test set and we convert that into a pandas data frame


In [20]:
#train_titles = split_titles(disp_title[0:4500]) #contains titles only
eval_titles  = split_titles(disp_title[4500:])
#train_text = split_text(disp_title[0:4500], abstracts[0:4500]) #this is the training set
eval_text = split_text(disp_title[4500:],abstracts[4500:])  #contains text + class
text_eval_df = pd.DataFrame(eval_text)


In [21]:
len(disp_title[4500:])

2609

In [22]:
from funcx.sdk.client import FuncXClient

#### Test the connection to the container running in a small desktop kub cluster.  
we already have the end point.

In [24]:
myep = 'd532a334-beaf-4aa1-a8e5-9a9594409152'

In [25]:
fxc = FuncXClient()

In [26]:
import time

In [27]:
import time
def funcx_imp(stringlist):
    import sys
    sys.path.append('/')
    from classifier import classifys
    x = classifys(stringlist)
    return x


In [28]:
s = text_eval_df[0][2604]
s

'We show that effective theories of matter that classically violate the null energy condition cannot be minimally coupled to Einstein gravity without being inconsistent with both string theory and black hole thermodynamics. We argue however that they could still be either non-minimally coupled or coupled to higher-curvature theories of gravity.'

In [29]:
go2_function = fxc.register_function(funcx_imp)

In [31]:
res = fxc.run([s], endpoint_id=myep, function_id=go2_function)
print(fxc.get_task(res)['pending'])
while fxc.get_task(res)['pending']:
    time.sleep(3)
    print(fxc.get_task(res)['pending'])
print (fxc.get_result(res))  

True
False
[2]


text_eval_df[1] is the column of the dataframe that give the correct answer.  in this case it is correct.

In [32]:
text_eval_df[1][2604]

2

#### run big tests
we will do the test in several batches because we aree using the classifys function which menans we are sending the entire text of each abstract.  we are starting with 1200 to 14500.

In [33]:
inlist = [text_eval_df[0][i] for i in range(1200,1450)]
chklist = [text_eval_df[1][i] for i in range(1200,1450)]

In [34]:
res = fxc.run(inlist, endpoint_id=myep, function_id=go2_function)
print(fxc.get_task(res)['pending'])
while fxc.get_task(res)['pending']:
    time.sleep(3)
    print(fxc.get_task(res)['pending'])
print (fxc.get_result(res))  

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
[1 2 2 2 4 2 0 4 1 2 2 2 1 1 2 2 1 2 2 2 1 2 2 1 0 2 0 3 2 1 4 2 1 0 3 1 0
 2 2 1 2 3 0 1 2 1 1 2 1 4 0 1 2 2 2 0 0 3 0 2 0 2 2 0 2 1 0 3 1 2 0 2 2 2
 3 0 3 0 3 2 2 2 0 1 4 2 4 2 3 0 2 1 2 2 2 0 3 0 4 1 1 2 1 2 1 3 0 2 2 2 1
 2 2 2 0 2 2 3 2 4 0 2 3 0 1 3 0 2 0 2 2 1 2 0 1 2 1 2 2 2 0 0 1 2 2 2 0 0
 3 2 0 1 2 2 4 0 3 4 0 2 1 2 2 3 2 2 3 0 2 2 0 4 2 0 0 2 2 1 2 2 2 1 2 1 2
 0 4 4 2 2 1 2 0 1 3 0 2 2 1 0 2 1 2 0 2 0 3 2 4 2 1 0 2 0 0 2 2 2 2 2 0 1
 0 1 2 1 2 3 0 4 2 2 1 0 2 0 0 3 2 1 0 2 2 4 0 3 3 0 1 2]


In [35]:
out = fxc.get_result(res)
correct = 0
for i in range(len(chklist)):
    if out[i] == chklist[i]:
        correct+=1
print(correct/len(chklist))

0.82


In [38]:
inlist = [text_eval_df[0][i] for i in range(1450,2604)]
chklist = [text_eval_df[1][i] for i in range(1450,2604)]
res = fxc.run(inlist, endpoint_id=myep, function_id=go2_function)
print(fxc.get_task(res)['pending'])
while fxc.get_task(res)['pending']:
    time.sleep(10)
    print(fxc.get_task(res)['pending'])
print (fxc.get_result(res))  

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
[2 2 2 ... 2 0 1]


In [39]:
out = fxc.get_result(res)
correct = 0
for i in range(len(chklist)):
    if out[i] == chklist[i]:
        correct+=1
print(correct/len(chklist))

0.7902946273830156


In [41]:
inlist = [text_eval_df[0][i] for i in range(0,1200)]
chklist = [text_eval_df[1][i] for i in range(0,1200)]
res = fxc.run(inlist, endpoint_id=myep, function_id=go2_function)
print(fxc.get_task(res)['pending'])
while fxc.get_task(res)['pending']:
    time.sleep(30)
    print(fxc.get_task(res)['pending'])
print (fxc.get_result(res))  

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
[0 1 0 ... 2 3 0]


In [42]:
out = fxc.get_result(res)
correct = 0
for i in range(len(chklist)):
    if out[i] == chklist[i]:
        correct+=1
print(correct/len(chklist))

0.7216666666666667


### compute average accuracy

In [44]:
(.82*250 - .79*(1450-2604) + .721*1200)/(250+(2604-1450)+1200)

0.7610829493087559

76% correct on a 1 out 5 choice test is not bad.   